## 生成2通道的重叠染色体图像

In [1]:
import os
import numpy as np
import cv2 
import math
import random

In [2]:
#存放单个染色体图片的文件夹路径
path='/home/xiang/Desktop/Share/ImgSeparateAndMix/chromosome_data/chromosome/F'

In [3]:
def rotate_bound(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    # grab the rotation matrix (applying the 
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
    # perform the actual rotation and return the imageb
#     angle=angle%90
    return cv2.warpAffine(image, M, (nW, nH),borderValue=(0,0,0))

In [4]:
def padding(img,height,width):
    (h,w)=img.shape[:2]
    a=height-h
    b=width-w
    if a>0 and b>0:
        an=np.zeros([a,w])
        img=np.r_[img,an]
        bn=np.zeros([height,b])
        img=np.c_[img,bn]
    elif a>0:
        an=np.zeros([a,w])
        img=np.r_[img,an]
    elif b>0:
        bn=np.zeros([h,b])
        img=np.c_[img,bn]
    return img

In [5]:
def make_Mask_And_Double(img1,img2):
    angle1=random.randint(0,360)
    angle2=random.randint(0,360)
    #如果角度不对，重新random
#     if not (abs(angle2%180-angle1%180)>70 and abs(angle1%180-angle2%180)<110):
#         continue
    img1=255-img1
    img2=255-img2
    img1=rotate_bound(img1,angle1)
    img2=rotate_bound(img2,angle2)
    img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    width=max(img1.shape[1],img2.shape[1])
    height=max(img1.shape[0],img2.shape[0])
    img1=padding(img1,height,width)
    img2=padding(img2,height,width)
    
    
    
    sum1=img1[img1!=0]
    sum2=img2[img2!=0]
    sumAnd=len(sum1)+len(sum2)
    arraryall=img1+img2
    sumUnion=len(arraryall[arraryall!=0])
    sumDiffer=sumAnd-sumUnion
    
    imgall=img1+img2
    sumUnion=len(imgall[imgall!=0])
    mask=np.zeros([height,width,2],dtype=np.uint8)
    double=np.zeros([height,width],dtype=np.uint8)
    #如果没有重叠，退出
    if sumDiffer==0:
        return False,mask,double
    
    for i in range(0,height):
        for j in range(0,width):
            #重叠部分
            if img1[i,j]!=0 and img2[i,j]!=0:
                mask[i,j,0]=255
                mask[i,j,1]=255
                sum=(int(img1[i,j]*0.7)+int(img2[i,j]*0.7))
                double[i,j]=255 if sum>255 else sum
                continue
            elif img1[i,j]!=0:
                mask[i,j,0]=255
            elif img2[i,j]!=0:
                mask[i,j,1]=255
            else:
                mask[i,j,0],mask[i,j,1]=0,0
            double[i,j]=img1[i,j]+img2[i,j]
    return True,mask,double

In [6]:
img_list=os.listdir(path)
for i,img_name in enumerate(img_list):
    if i==10:
        break
    a=random.randint(0,len(img_list)-1)  
    b=random.randint(0,len(img_list)-1)
    img1=cv2.imread(os.path.join(path,img_list[a]))
    img2=cv2.imread(os.path.join(path,img_list[b]))
    bo,mask,imgDouble=make_Mask_And_Double(img1,img2)
    if not bo:
        continue
    imgDouble = cv2.GaussianBlur(imgDouble,(3,3),0)
    cv2.imwrite('mask_and_double/{}_Double.png'.format(i),imgDouble)
    cv2.imwrite('mask_and_double/{}_mask1.png'.format(i),mask[:,:,0])
    cv2.imwrite('mask_and_double/{}_mask2.png'.format(i),mask[:,:,1])
    print(i)
#     cv2.imshow('img',imgDouble)
#     cv2.imshow('img1',mask[:,:,0])
#     cv2.imshow('img2',mask[:,:,1])
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
#     break

0
2
3
4
5
6
7
8
9
